<center>

**Vježba 2 - Osnovni IRIS model,MLflow + SC2 na srcu**

</center>

Iskoristimo ovu staru vježbu na okolini na SRCU da pokažemo kako raditi automatski deployment modela!

Prvo učitavamo potrebne python module.

In [ ]:
from sklearn import datasets
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
import pickle

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

Prvo učitamo IRIS skup podataka iz *scikit-learn* modula:
- Učitamo skup podataka, te ga razdvojimo na ulaze i labele
- Razdvojimo IRIS skup podataka na podskup za učenje i testiranje

In [ ]:
def loadIrisDataset():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target
    return (iris,X,y)

(iris,X,y)=loadIrisDataset()
display(X)

X_in = pd.DataFrame(X, columns=iris.feature_names)

X_train, X_test, y_train, y_test = train_test_split(X_in, y, test_size = 0.2)

Postavimo *MLflow* server URL - sada nam treba autentifikacija. Zatim potražimo naš IRIS eksperiment.  
U slučaju da eksperiment nije pronađen, stvorimo ga.

In [ ]:
import os
os.environ['MLFLOW_TRACKING_USERNAME']='team1'
os.environ['MLFLOW_TRACKING_PASSWORD']='iKAMiMhK2i'
mlflow.set_tracking_uri("https://mlflow-team1.aris.fer.hr")
exps = mlflow.search_experiments(filter_string="name LIKE '%Iris%'")
if len(exps)==0:
    exp_id = mlflow.create_experiment("MLP - Iris dataset - 3")
else:
    exp_id = exps[0].experiment_id

Izgenerirat ćemo tri modela. Jedan s scikit-learn cjevovodom u kojem ćemo imati slijed scaler->mlp.  
A druga dva modela će biti: posebno scaler i posebno MLP, koje ćemo kasnije onda povezati u SC2 cjevovod.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%bash
mkdir ./iris-model
mkdir ./iris-model/scaler
mkdir ./iris-model/mlpraw

In [ ]:
%%writefile ./iris-model/scaler/model-settings.json
{
    "name": "mlp-iris-scaler",
    "implementation": "mlserver_sklearn.SKLearnModel",
    "parameters": {
        "uri": "./model.pkl",
        "version": "v1.0.0",
        "extra": {
            "predict_fn": "transform"
        }
    }
}

In [ ]:
%%writefile ./iris-model/mlpraw/model-settings.json
{
    "name": "mlp-iris-scaler",
    "implementation": "mlserver_sklearn.SKLearnModel",
    "parameters": {
        "uri": "./model.pkl",
        "version": "v1.0.0"
    }
}

Naučimo naše modele i spremimo ih i MLflow.

In [ ]:
from sklearn.pipeline import Pipeline

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000, verbose=1)

pipe=Pipeline([('scaler',StandardScaler()),('mlp',mlp)])
pipe.fit(X_train, y_train)

scaler=StandardScaler()
scaler.fit(X_train)

mlp2 = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000)
mlp2.fit(X_train,y_train)

schema = infer_signature(X_train, y_train)
with mlflow.start_run(run_name="Training iris MLP model",
                        experiment_id=exp_id) as run:
    mlflow.sklearn.log_model(pipe, artifact_path="sklearn-model", signature=schema)
    mlflow.sklearn.log_model(scaler, artifact_path="sklearn-scaler", signature=schema)
    mlflow.sklearn.log_model(mlp2, artifact_path="sklearn-raw-model", signature=schema)
    mlflow.log_artifact('./iris-model/scaler/model-settings.json', artifact_path="sklearn-scaler")
    mlflow.log_artifact('./iris-model/mlpraw/model-settings.json', artifact_path="sklearn-raw-model")
        
name1="Iris MLP model-mpipeline"
name2="Iris MLP model-scaler"
name3="Iris MLP model-rawmodel"

client = MlflowClient()
rms = client.search_registered_models("name='"+name1+"'")
if len(rms)==0: client.create_registered_model(name1)

model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
mv = client.create_model_version(name1, model_src, run.info.run_id, 
    description="Pipeline for iris dataset")

rms = client.search_registered_models("name='"+name2+"'")
if len(rms)==0: client.create_registered_model(name2)

model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
mv = client.create_model_version(name2, model_src, run.info.run_id, 
    description="Scaler for iris dataset")

rms = client.search_registered_models("name='"+name3+"'")
if len(rms)==0: client.create_registered_model(name3)

model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
mv = client.create_model_version(name3, model_src, run.info.run_id, 
    description="MLP for iris dataset")

mlflow.end_run()

Koristimo MLflow tip modela... Pripazite na shemu koju smo preuzeli od skupa podataka.

In [ ]:
%%writetemplate iris.yaml
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "s3://team1/mlflow/{exp_id}/{run.info.run_id}/artifacts/sklearn-model"
  secretName: "minio"
  requirements:
  - mlflow

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

inference_request = {
    "id": 56,
    "inputs": [       
          {
            "name": "sepal length (cm)",
            "shape": [3],
            "datatype": "FP32",
            "data": [0.432165, -1.143017, 6.2],
          },
          {
            "name": "sepal width (cm)",
            "shape": [3],
            "datatype": "FP32",
            "data": [0.788808, 1.249201, 3.4],
          },
          {
            "name": "petal length (cm)",
            "shape": [3],
            "datatype": "FP32",
            "data": [0.933271, -1.340227, 5.4],
          },
          {
            "name": "petal width (cm)",
            "shape": [3],
            "datatype": "FP32",
            "data": [1.448832, -1.447076, 2.3],
          }
    ]
}

endpoint = "https://seldon-mesh-team1.aris.fer.hr/v2/models/iris/infer"
response = requests.post(endpoint, json=inference_request, auth=HTTPBasicAuth('team1', 'iKAMiMhK2i'))

response.json()



Na SC2 instaliramo dva modela: scaler (transformer) i model (MLP)

In [ ]:
%%writetemplate iris-scaler.yaml
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris-scaler
spec:
  storageUri: "s3://team1/mlflow/{exp_id}/{run.info.run_id}/artifacts/sklearn-scaler"
  secretName: "minio"
  requirements:
  - sklearn

In [ ]:
%%writetemplate iris-raw-mlp.yaml
apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris-raw-mlp
spec:
  storageUri: "s3://team1/mlflow/{exp_id}/{run.info.run_id}/artifacts/sklearn-raw-model"
  secretName: "minio"
  requirements:
  - sklearn

Probamo čisti MLP, bez skaliranja. Provjerimo da li su rezultati isti?

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

inference_request = {
    "id": 57,
    "inputs": [
        {
          "name": "predict",
          "shape": (3,4),
          "datatype": "FP32",
          "data": [[-1.143017, 1.249201, -1.340227, -1.447076],[0.432165, 0.788808, 0.933271, 1.448832],[6.2, 3.4, 5.4, 2.3]]
        }
    ]
}

endpoint = "https://seldon-mesh-team1.aris.fer.hr/v2/models/iris-raw-mlp/infer"
response = requests.post(endpoint, json=inference_request, auth=HTTPBasicAuth('team1', 'iKAMiMhK2i'))

response.json()



Povežemo dva modela na SC2 u cjevovod.

In [ ]:
%%writefile iris-pipeline.yaml
apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: iris-pipeline
spec:
  steps:
    - name: iris-scaler
    - name: iris-raw-mlp
      inputs:
      - iris-scaler
  output:
    steps:
    - iris-raw-mlp

I sad pozovemo cjevovod. Ovaj puta se vidi da imamo skaliranje koje nas vraća na ispravne rezultate.

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": (3,4),
          "datatype": "FP32",
          "data": [[-1.143017, 1.249201, -1.340227, -1.447076],[0.432165, 0.788808, 0.933271, 1.448832],[6.2, 3.4, 5.4, 2.3]]
        }
    ]
}

endpoint = "https://seldon-mesh-team1.aris.fer.hr/v2/pipelines/iris-pipeline/infer"
response = requests.post(endpoint, json=inference_request, auth=HTTPBasicAuth('team1', 'iKAMiMhK2i'))

response.json()


Ne zaboravite provjeriti kako monitoring radi. Spojite se na Grafanu od okolini na srcu i promatrajte što se dešava kad radite pozive na model.